In [1]:
!pip install accelerate>=0.20.1

# 필요한 모듈 install

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
 !pip install transformers
 !pip install pytorch-lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 62.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.0/727.0 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 764.8/764.8 kB 13.9 MB/s eta 0:00:00


In [4]:
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
)
import torch
import torch.nn as nn

# 데이터 다운로드 - 데이터 개수 100개

In [5]:
import pandas as pd
train_df = pd.read_csv('/content/drive/MyDrive/data/news_summary_train_dataset.tsv', sep='\t')
df = train_df[:100]

# 모델, tokenizer 설정

In [6]:
model_name = 'digit82/kobart-summarization'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


# 데이터 전처리
- target 데이터에 bos, eos 붙이기
- max_length : 1024

In [7]:
input_tokenized = []
target_tokenized = []
for i in range(len(df)):

    input_text = df.iloc[i, 0]
    target_text = df.iloc[i, 1]

    input_ids = tokenizer(input_text, padding="max_length", max_length=1024, truncation=True)["input_ids"]
    target_ids = tokenizer('<s>' + target_text + '</s>', padding="max_length", max_length=1024, truncation=True)["input_ids"]

    input_tokenized.append(input_ids)
    target_tokenized.append(target_ids)

In [8]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, input_tokenized, target_tokenized):
        self.input_tokenized = torch.LongTensor(input_tokenized)
        self.target_tokenized = torch.LongTensor(target_tokenized)

    def __len__(self):
        return len(self.input_tokenized)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_tokenized[idx],
            'labels': self.target_tokenized[idx]
        }

In [9]:
input_train, input_test, output_train, output_test = input_tokenized[:90], input_tokenized[90:100], target_tokenized[:90], target_tokenized[90:100]

In [18]:
# from torch.utils.data import DataLoader

# train_dataset = CustomDataset(input_train, output_train)
# test_dataset = CustomDataset(input_test,output_test)

# dl_train = DataLoader(train_dataset, batch_size=5, shuffle=False)
# dl_test = DataLoader(test_dataset, batch_size=5, shuffle=False)

train_dataset = CustomDataset(input_train, output_train)
test_dataset = CustomDataset(input_test,output_test)

In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [11]:
torch.cuda.empty_cache()

In [16]:
import torch
# from transformers import PreTrainedTokenizerFast
# from transformers import BartForConditionalGeneration
# from transformers import BartConfig

# model_name = 'digit82/kobart-summarization'
# config = BartConfig.from_pretrained(model_name)
# config.decoder_layers = 2

# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name, config = config)
for param in model.get_encoder().parameters():
    param.requires_grad = False

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer, model=model
)


model

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=

In [29]:
model_path = "/content/drive/MyDrive/hong/model/"

training_args = Seq2SeqTrainingArguments(
    output_dir=model_path, #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=5, # number of training epochs
    per_device_train_batch_size=9, # batch size for training
    per_device_eval_batch_size=10,  # batch size for evaluation
    eval_steps=12, # Number of update steps between two evaluations.
    save_steps=1000, # after # steps model is saved
    logging_steps=10,
    warmup_steps=300,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    evaluation_strategy="steps",
    save_total_limit=3
    )

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [30]:
result = trainer.train()

Step,Training Loss,Validation Loss
12,4.582600,4.588061
24,4.580900,4.585445
36,4.578000,4.581874
48,4.574200,4.578203


In [32]:
result.training_loss

4.576835479736328

## 학습 돌리기

In [14]:
model = model.to(device)

from tqdm import tqdm
epochs = 200
loss = nn.CrossEntropyLoss().to(device)
optim = torch.optim.Adam(params=model.parameters(), lr=0.001)

epoch_train_loss_list = []
epoch_val_loss_list= []

for epoch in tqdm(range(epochs)):
    epoch_loss = 0.0
    for batch in dl_train:
        input, output = batch['input_ids'].to(device), batch['labels'].to(device)
        pred = model(input, output)
        batch_loss = 0.0
        batch_loss += loss(pred[0].transpose(1, 2), output)

        epoch_loss += batch_loss

    epoch_loss = epoch_loss/len(dl_train)

    epoch_loss.append(epoch_loss.item())

    optim.zero_grad()
    epoch_loss.backward()
    optim.step()

    if (epoch+1) % 10 == 0:
        print('Epoch:', '%02d' % (epoch + 1), 'cost =', '{:.6f}'.format(epoch_loss))

    # optim.zero_grad()
    # epoch_loss.backward()
    # optim.step()

    with torch.no_grad():

        epoch_val_loss = 0.0
        for batch_val in dl_test:
            input_val, output_val = batch_val['input_ids'].to(device), batch_val['labels'].to(device)
            pred_val = model(input_val)
            batch_loss_val = 0.0
            batch_loss_val += loss(pred_val[0].transpose(1, 2), output_val)

            epoch_val_loss += batch_loss_val

        epoch_val_loss = epoch_val_loss/len(dl_test)

        epoch_val_loss_list.append(epoch_val_loss.item())
        if (epoch+1) % 10 == 0:
            print('validation loss =', '{:.6f}'.format(epoch_val_loss))

  0%|          | 0/200 [00:04<?, ?it/s]


OutOfMemoryError: ignored

## ACCURACY찍기!

In [ ]:
# 100개 데이터 모델로 돌린 결과 리스트에 담기 : 담긴건 토큰들
result = []
for i in range(100):
    input_ids = tokenizer(df.iloc[i, 0], return_tensors="pt")["input_ids"]
    output = model.generate(input_ids.to(device))
    result.append(output[0])

In [ ]:
# label토큰에서 pad빼기 귀찮으니까 그냥 다시 토큰화 해버리기, special token없이
target_tokenized = []
for i in range(len(df)):
    target_text = df.iloc[i, 1]
    target_ids = tokenizer(target_text)["input_ids"]
    target_tokenized.append(target_ids)

In [ ]:
int(False)

In [ ]:
# 결과물에는 [2, 0, ..., 1] 이런식으로 다 되어있어서 스페셜토큰 다 빼주려고 슬라이싱해서 없어버렸습니다. 그리고 반환타입이 텐서인데 그냥 리스트로 바꿔버림
result_new = []
for i in range(100):
    result_new.append(result[i].tolist()[2:-1])

result_new[2]

In [ ]:
# count는 예측이랑 정답의 같은 위치에 같은 토큰이 있을경우 +1
# total은 정답 토큰 총 개수
count = 0
total = 0
for i in range(100):
    total += len(target_tokenized[i])
    for j in range(len(result_new[i])):
        count += int(result_new[i][j] == target_tokenized[i][j])

print(count/total)

In [ ]:
348/12

In [ ]:
count

In [ ]:
total

In [ ]:
predict_text = []
for i in range(100):
    predict_text.append(tokenizer.decode(result_new[i]))

In [ ]:
df_result = pd.DataFrame({"predict":predict_text})
df_result['labels'] = df.iloc[:, 1]

In [ ]:
df_result.head(10)

In [ ]:
result

In [ ]:
for i in range(100):
    if len(result)

In [ ]:
tokenizer.decode(output[0], skip_special_tokens=True)

In [ ]:
df.iloc[0, 1]

In [ ]:
torch.cuda.empty_cache()

In [ ]:
trainer.train()

In [ ]:
torch.save(model, "/content/drive/MyDrive/hong/models/model_summary.pt")

In [ ]:
input_ids = tokenizer(df.iloc[0, 0], return_tensors="pt")["input_ids"]

generated_ids = model.generate(input_ids.to(device))
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

In [ ]:
generated_text

In [ ]:
input_ids = tokenizer(df.iloc[1, 0], return_tensors="pt")["input_ids"]

generated_ids = model.generate(input_ids.to(device))
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
generated_text

In [ ]:
result = []
labels = df.iloc[:, 1]
for i in range(100):
    input_ids = tokenizer(df.iloc[i, 0], return_tensors="pt")["input_ids"]

    generated_ids = model.generate(input_ids.to(device))
    # generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    result.append(generated_ids)

In [ ]:
df_tmp = pd.DataFrame({'result':result})
df_tmp['label'] = labels
df_tmp

In [ ]:
generated_ids

In [ ]:
result[4][0]

In [ ]:
result[0][0][0].item()

In [ ]:
target_tokenized[0]

In [ ]:
while 3 in

In [ ]:
count = 0

for i in range(100):
    for j in range(len(result[i])):
        if result[i][0][j].item() == target_tokenized[i][j]:
            count += 1

In [ ]:
count

In [ ]:
model_path = "/content/drive/MyDrive/hong/models/"

training_args = Seq2SeqTrainingArguments(
    output_dir=model_path, #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=5, # number of training epochs
    per_device_train_batch_size=64, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps=500, # Number of update steps between two evaluations.
    save_steps=1000, # after # steps model is saved
    warmup_steps=300,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    evaluation_strategy="steps",
    save_total_limit=3
    )

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
trainer.train()

In [ ]:
checkpoint = torch.load("/content/drive/MyDrive/hong/models/checkpoint-3000/pytorch_model.bin")
model.load_state_dict(checkpoint)

In [ ]:
model = model.to(device)

In [ ]:
input_ids = tokenizer(df.iloc[120, 0], return_tensors="pt")["input_ids"]

generated_ids = model.generate(input_ids.to(device))
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

In [ ]:
generated_text

In [ ]:
generated_text

In [ ]:
df.iloc[120, 1]

In [ ]:
generated_text

In [ ]:
df.iloc[110, 1]

In [ ]:
input_ids = tokenizer(df.iloc[110, 0], return_tensors="pt")["input_ids"]

generated_ids = model.generate(input_ids.to(device))
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

In [ ]:
from transformers import BartConfig
import torch.nn as nn

class SummarizeModel(nn.Module):
    def __init__(self, model_name = 'hyunwoongko/kobart', num_decode_layers = 1):
        super(SummarizeModel, self).__init__()
        self.config = BartConfig.from_pretrained(model_name)
        self.config.decoder_layers = num_decode_layers
        self.bart =  BartForConditionalGeneration.from_pretrained(model_name, config = self.config)
        for param in self.bart.get_encoder().parameters():
                param.requires_grad = False

    def forward(self, input_ids, dec_input):
        result = self.bart(input_ids, dec_input)

        return result[0]


    def generate(self, input_ids):
        result = self.bart(input_ids)

        return result[0]

In [ ]:
model = SummarizeModel().to(device)

In [ ]:
model

In [ ]:
result = model.generate(next(iter(dl_train))['input_ids'].to(device))

In [ ]:
result.shape

In [ ]:
# from tqdm import tqdm
# epochs = 100
# loss = nn.CrossEntropyLoss().to(device)
# optim = torch.optim.Adam(params=model.parameters(), lr=0.001)

# for epoch in tqdm(range(epochs)):
#     epoch_loss = 0.0
#     for batch in dl_train:
#         input, output = batch['input_ids'].to(device), batch['labels'].to(device)
#         pred = model(input, output)
#         pred = pred.transpose(0, 1)
#         output = output.transpose(0, 1)
#         batch_loss = 0.0
#         for i in range(512):  # input_size만큼 돌기
#             batch_loss += loss(pred[i], output[i])
#         batch_loss = batch_loss/512

#         epoch_loss += batch_loss

#     epoch_loss = epoch_loss/len(dl_train)
#     if (epoch+1) % 10 == 0:
#       print('Epoch:', '%02d' % (epoch + 1), 'cost =', '{:.6f}'.format(epoch_loss))

#     optim.zero_grad()
#     epoch_loss.backward()
#     optim.step()

#     with torch.no_grad():
#         epoch_val_loss = 0.0
#         for batch_val in dl_test:
#             input_val, output_val = batch_val['input_ids'].to(device), batch_val['labels'].to(device)
#             pred_val = model.generate(input_val)
#             pred_val = pred_val.transpose(0, 1)
#             output_val = output_val.transpose(0, 1)

#             batch_loss_val = 0.0
#             for i in range(512):
#                 batch_loss_val += loss(pred_val[i], output_val[i])

#             batch_loss_val = batch_loss_val/512

#         epoch_val_loss = epoch_val_loss/len(dl_test)
#         if (epoch+1) % 10 == 0:
#             print('validation loss =', '{:.6f}'.format(batch_loss_val))

In [ ]:
torch.save(model, "/content/drive/MyDrive/hong/model_simple.pt")